In [1]:
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.warehouse import Warehouse

import pandas as pd

In [2]:
## Create a session using our config file
session = Session.builder.config("connection_name","myconnection").create()
root = Root(session)

c:\Users\powel\anaconda3\envs\.venv-sp500-stocks-app\lib\site-packages\snowflake\connector\config_manager.py:344: UserWarning: Bad owner or permissions on ..\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


In [3]:
## Create a New or Update an Existing Warehouse
investment_wh = Warehouse(
  name="investment_wh",
    warehouse_size="SMALL",
    auto_suspend=300,
    auto_resume='true'
)
investment_wh_res = root.warehouses["investment_wh"]
investment_wh_res.create_or_update(investment_wh)

In [4]:
## Create or replace Database in the account
database = root.databases.create(Database(name="INVESTMENT_DB"), mode="orreplace")

In [5]:
## Create or replace Schema in a Database
schema = database.schemas.create(Schema(name="INVESTMENT_SCHEMA"), mode="orreplace")

### Load sp500_returns dataset into Snowflake

In [6]:
## Load in our returns CSV file to a pandas DataFrame
sp500_returns_pd =  pd.read_csv("../csv_dumps/sp500_returns_time_series.csv")
sp500_returns_pd.tail()

,DATE,TICKER,RETURN,RETURN_SP500,EXCESS_RETURN
501486,2023-12-11,ZTS,0.026327,0.003924,0.022403
501487,2023-12-12,ZTS,0.010609,0.004599,0.006010
501488,2023-12-13,ZTS,0.031023,0.013651,0.017372
501489,2023-12-14,ZTS,0.013576,0.002647,0.010929
501490,2023-12-15,ZTS,-0.018991,-0.000076,-0.018915


In [7]:
## Create a Snowpark DataFrame object from the pandas DataFrame
sp500_returns_df = session.create_dataframe(sp500_returns_pd)
sp500_returns_df.show()

-------------------------------------------------------------------------------------------
|"DATE"      |"TICKER"  |"RETURN"             |"RETURN_SP500"       |"EXCESS_RETURN"      |
-------------------------------------------------------------------------------------------
|2020-01-02  |A         |NULL                 |NULL                 |NULL                 |
|2020-01-03  |A         |-0.0160558625376078  |-0.0070598705620453  |-0.0089959919755624  |
|2020-01-06  |A         |0.0029560971447193   |0.0035333728907271   |-0.0005772757460078  |
|2020-01-07  |A         |0.0030655935136134   |-0.0028032386528962  |0.0058688321665096   |
|2020-01-08  |A         |0.0098730003124209   |0.0049024513689781   |0.0049705489434428   |
|2020-01-09  |A         |0.0157120164791473   |0.0066552626054889   |0.0090567538736583   |
|2020-01-10  |A         |0.0036669906801432   |-0.0028551786896625  |0.0065221693698057   |
|2020-01-13  |A         |-0.0014838309342424  |0.0069762152525698   |-0.00846004

In [8]:
## Save the Snowpark DataFrame to a Snowflake table in our account
sp500_returns_df.write.mode("overwrite").save_as_table("sp500_returns")

### Load sp500_info dataset into Snowflake

In [9]:
## Load in our info CSV file to a pandas DataFrame
sp500_info_pd = pd.read_csv("../csv_dumps/sp500_info.csv")
sp500_info_pd.tail()

,SYMBOL,ADDRESS1,CITY,STATE,ZIP,COUNTRY,PHONE,WEBSITE,INDUSTRY,INDUSTRYKEY,...,PEGRATIO,TARGETHIGHPRICE,TARGETLOWPRICE,TARGETMEANPRICE,TARGETMEDIANPRICE,RECOMMENDATIONMEAN,NUMBEROFANALYSTOPINIONS,EARNINGSGROWTH,FAX,INDUSTRYSYMBOL
498,YUM,1441 Gardiner Lane,Louisville,KY,40213,United States,502 874 8300,https://www.yum.com,Restaurants,NaN,...,1.79,160.0,116.0,138.10,136.0,2.4,21.0,0.281,NaN,NaN
499,ZBRA,3 Overlook Point,Lincolnshire,IL,60069,United States,847 634 6700,https://www.zebra.com,Communication Equipment,NaN,...,6.08,340.0,183.0,265.67,275.0,2.4,9.0,NaN,NaN,NaN
500,ZBH,345 East Main Street,Warsaw,IN,46580,United States,574 373 3121,https://www.zimmerbiomet.com,Medical Devices,NaN,...,2.28,150.0,99.0,129.38,130.0,2.6,24.0,-0.163,NaN,NaN
501,ZION,One South Main Street,Salt Lake City,UT,84133-1109,United States,801 844 7637,https://www.zionsbancorporation.com,Banks—Regional,NaN,...,-0.31,48.0,32.0,37.95,37.5,2.7,20.0,-0.191,NaN,NaN
502,ZTS,10 Sylvan Way,Parsippany,NJ,07054,United States,973 822 7000,https://www.zoetis.com,Drug Manufacturers—Specialty & Generic,NaN,...,3.53,255.0,190.0,216.18,215.0,1.9,11.0,0.142,NaN,NaN


In [10]:
## Create a Snowpark DataFrame object from the pandas DataFrame
sp500_info_df = session.create_dataframe(sp500_info_pd)
sp500_info_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
## Save the Snowpark DataFrame to a Snowflake table in our account
sp500_info_df.write.mode("overwrite").save_as_table("sp500_info")